# Typeface comparisons

- comparison of responses for different typefaces (Fisher’s Exact test)

In [2]:
# intro, defaults (shared across notebooks)

import os
import sys

import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
from chardict import chardict
%matplotlib inline


# ---------------------------------------------------
# Fisher’s Exact test via R

from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
# activate the Pandas conversion of rpy2
pandas2ri.activate()
# import stats package
rstats = importr("stats")

def fisher_exact(df):
    """
    (Using R via rpy2. See the imports above.)
    
    Performs Fisher’s exact test for testing the null of independence
    of rows and columns in a contingency table with fixed marginals.
    
    Input:  df - DataFrame 2x3
    Output: pval - p-value of the test
    """
    
    # convert DataFrame to R DataFrame
    #rdf = pandas2ri.py2ri(df)
    # perform Fisher’s test
    res = rstats.fisher_test(df)
    # return p-value
    return res.rx("p.value")[0][0]


# ---------------------------------------------------
# helpers

def fix_columns(cols):
    for col in cols:
        if col[0] == "control":
            yield col
        else:
            col = list(col)
            col[-1] = tuple(eval(col[-1]))
            yield tuple(col)

def make_compact(d):
    """
    Make DataFrame compact = use [0, 1, 2]
    instead of all characters on the index
    """

    d_compact = pd.DataFrame(columns=d.columns, index=[0, 1, 2])
    for col in d.columns:
        triplet = col[-1] # last item in the index is a triplet
        col_data = pd.Series(d[col].dropna(), index=triplet)
        d_compact[col] = list(col_data) # past regardless the index
    d = d_compact.fillna(0)
    return d


# ---------------------------------------------------
# set global properties for plotting

font = {"family":"Adapter Mono PE", "size":"10", "weight":"medium"}
figure = {"titlesize":"10","titleweight":"medium"}
axes = {"titlesize":"10", "titleweight":"medium", "labelsize":"10", "labelweight":"medium"}
plt.rc("font", **font)
plt.rc("figure", **figure)
plt.rc("axes", **axes)
box_colors = dict(boxes="Black", whiskers="0.5", medians="Black", caps="0.5")

In [3]:
# Get data

# raw data

d = pd.read_csv("csv/raw-data-preprocessed.csv", index_col=0, header=[0,1,2], dtype="unicode")
non_control_columns = [col for col in d.columns if col[0] != "control"]

# fix type
d["control", "control", "order"] = d["control", "control", "order"].astype("float").astype("int")

# fix triplet columns (convert them to tuples)
d.columns = pd.MultiIndex.from_tuples(list(fix_columns(d.columns)))

# data frame just for the demographics
demo = d["control", "control"]

print("Imported %d rows, %d columns." % d.shape)

# frequencies

global_frequencies = pd.read_csv("csv/frequencies/frequencies_all.csv", index_col=0, header=[0, 1, 2])
global_counts = pd.read_csv("csv/counts/counts_all.csv", index_col=0, header=[0, 1, 2])

# Warning: the third level (2) of the columns is now strings: '('cyrl.ef', 'cyrl.er', 'cyrl.yu')', not tuples of strings: ('cyrl.ef', 'cyrl.er', 'cyrl.yu')

print("Imported %d frequencies and %d counts." % (len(global_frequencies), len(global_counts)))


Imported 1787 rows, 1579 columns.
Imported 96 frequencies and 96 counts.


/var/folders/y9/cdnw18g5777bz8cqsp6zgd380000gn/T/ipykernel_84385/3137323641.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  demo = d["control", "control"]


In [4]:
# Comparison of responses for different typefaces

# get a list of triplets shared by at least two typefaces
# using only sets with 5 or more triplets
# those exist only in Latin (4 typefaces)
# some smaller sets are in Devanagari and Cyrillic, not using those

# using Fisher’s exact test to compare results between two different groups
# if the p-value is smaller than 0.05 then there is an effect (i.e. the results are dependent on the typeface)
# requires imports and activation of rpy2 (see the first cell)

def scale_counts(counts, total):
    """
    Scale the counts to fit the total provided.
    Avoid rounding errors.
    """
    
    counts = counts / counts.sum() * total
    counts = counts.round()
    counts.iloc[2] = total - (counts.iloc[0] + counts.iloc[1])
    return counts


for script in ["cyrillic", "devanagari", "latin"]:
    # find typefaces with shared triplets
    typefaces_triplets = {}
    f = global_frequencies[script]
    f.columns = f.columns.remove_unused_levels()
    typefaces = sorted(f.columns.levels[0])
    for i, t1 in enumerate(typefaces):
        triplets1 = set(f[t1].columns)
        for j in range(i+1, len(typefaces)):
            t2 = typefaces[j]
            triplets2 = set(f[t2].columns)
            overlap = list(triplets1 & triplets2)
            if len(overlap) > 0:
                typefaces_triplets[(t1, t2)] = overlap

    # make dirs
    if not os.path.exists("csv/typeface-comparisons/"):
        os.makedirs("csv/typeface-comparisons/")

    # report comparisons
    for t1, t2 in typefaces_triplets:
        # make MultiIndex: triplet | typefaces
        for triplet in typefaces_triplets[(t1, t2)]:
            triplet_ = eval(triplet)
            report = pd.DataFrame(columns=[t1, t2], index=[*triplet_, "p-value", "effect"])
            # get response counts for both typefaces for a triplet
            # simplify the column names so R does not have a problem
            counts = pd.DataFrame(columns=["T1","T2"], index=triplet_)
            # fill NAs with 0 so we FE test can be calculated
            # get only the values for the three characters from the triplet
            counts["T1"] = global_counts[script, t1, triplet]
            counts["T2"] = global_counts[script, t2, triplet]
            counts = counts.fillna(0)
            # scale counts with the smaller total to the larger total
            total1 = counts["T1"].sum()
            total2 = counts["T2"].sum()
            if total1 < total2:
                counts["T1"] = scale_counts(counts["T1"], total2)
            elif total1 > total2:
                counts["T2"] = scale_counts(counts["T2"], total1)
            # run FE
            p_val = fisher_exact(counts)
            report[t1][0:3] = counts["T1"] 
            report[t2][0:3] = counts["T2"] 
            report[t1]["p-value"] = round(p_val, 3)
            report[t2]["p-value"] = round(p_val, 3)
            report[t1]["effect"] = (p_val < 0.05) # alpha
            report[t2]["effect"] = (p_val < 0.05) # alpha

            print("Saving report for comparison", t1, "vs", t2, triplet)
            path = os.path.join("csv", "typeface-comparisons", script, "%s__%s__%s.csv" % (t1, t2, "_".join(triplet_)))
            dirs = os.path.dirname(path)
            if not os.path.exists(dirs):
                os.makedirs(dirs)
            report.to_csv(path)

Saving report for comparison century-schoolbook vs courier-new ('cyrl.de', 'cyrl.ka', 'cyrl.ya')
Saving report for comparison pt-sans vs pt-serif ('cyrl.e', 'cyrl.ha', 'cyrl.te')
Saving report for comparison devanagari-mt vs nirmala-ui ('deva.Cha', 'deva.Dha', 'deva.U')
Saving report for comparison devanagari-mt vs nirmala-ui ('deva.Cha', 'deva.Dha', 'deva.Tta')
Saving report for comparison devanagari-mt vs nirmala-ui ('deva.Cha', 'deva.Tta', 'deva.U')
Saving report for comparison devanagari-mt vs nirmala-ui ('deva.Dha', 'deva.Tta', 'deva.U')
Saving report for comparison lohit-devanagari vs murty-hindi ('deva.Nna', 'deva.Tha', 'deva.Ya')
Saving report for comparison arial vs times-new-roman ('latn.a', 'latn.l', 'latn.t')
Saving report for comparison calibri vs candara ('latn.i', 'latn.j', 'latn.v')
Saving report for comparison calibri vs candara ('latn.d', 'latn.i', 'latn.j')
Saving report for comparison calibri vs candara ('latn.d', 'latn.i', 'latn.v')
Saving report for comparison cal

In [6]:
# Compare distinctiveness tables

from scipy import stats

for t1, t2 in [("candara", "georgia"), ("futura", "pt-serif")]:
    d1 = pd.read_csv("csv/distinctiveness-tables/latin/%s.csv" % t1, index_col=[0, 1], header=0)
    d1 /= d1.sum()
    d2 = pd.read_csv("csv/distinctiveness-tables/latin/%s.csv" % t2, index_col=[0, 1], header=0)
    d2 /= d2.sum()
    display(d1.compare(d2))
    print(t1, "x", t2)
    print("Spearman’s rank correlation coefficient: %.2f, p-value: %.3f" % stats.spearmanr(d1, d2, nan_policy="omit"))
    

0          
              self     other
latn.b b  0.117241  0.115789
latn.d d  0.120690  0.112281
latn.i i  0.137931  0.133333
latn.j j  0.120690  0.133333
latn.k k  0.103448  0.098246
latn.l l  0.127586  0.098246
latn.v v  0.144828  0.164912
latn.y y  0.127586  0.143860

candara x georgia
Spearman’s rank correlation coefficient: 0.65, p-value: 0.083


0          
              self     other
latn.g g  0.129371  0.210909
latn.h h  0.083916  0.072727
latn.m m  0.083916  0.058182
latn.n n  0.087413  0.058182
latn.q q  0.118881  0.138182
latn.r r  0.094406  0.098182
latn.y y  0.185315  0.160000
latn.z z  0.216783  0.203636

futura x pt-serif
Spearman’s rank correlation coefficient: 0.87, p-value: 0.005
